# Riot APi로 유저정보 받기

In [1]:
import requests
from urllib import parse
import json
import pandas as pd
from urllib.request import urlopen

In [2]:
key_fd = open('../keys/api_key.txt', mode='r')
api_key = key_fd.read(100)
key_fd.close()

In [3]:
request_headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com",
    "X-Riot-Token": api_key  #Riot API 키   
}

In [4]:
champion_df = pd.read_csv('../00. data/champions.csv', index_col = 0)
champion_df.head()

,name,key,tags
0,아트록스,266,"Fighter, Tank"
1,아리,103,"Mage, Assassin"
2,아칼리,84,Assassin
3,알리스타,12,"Tank, Support"
4,아무무,32,"Tank, Mage"


In [5]:
item_df = pd.read_csv('../00. data/items.csv', index_col = 0)
item_df.head()

,name,key,gold,tags
0,장화,1001,300,Boots
1,요정의 부적,1004,250,ManaRegen
2,원기 회복의 구슬,1006,150,HealthRegen
3,거인의 허리띠,1011,900,Health
4,민첩성의 망토,1018,600,CriticalStrike


In [6]:
# 유저의 암호화된 아이디를 불러오는 함수 유저의 암호화된 아이디를 불러오는 함수
def searchUserId(nickname):
    user_data = requests.get("https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/" + parse.quote(nickname), headers=request_headers).json()
    userId = user_data['id']
    userAccountId = user_data['accountId']
    userPuuid = user_data['puuid'] 
    return userId, userAccountId, userPuuid
# 한유저의 특정 챔피언 매치ID들을 불러오는 함수
def searchChampMatchId(accountId, champion):
    championKey = champion_df['key'][champion_df['key'][champion_df['name'] == champion].index[0]] # 챔피언 키구하기
    match_data = requests.get("https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/" + accountId + "?champion=" + str(championKey) + '&queue=420',headers=request_headers).json()['matches']
    match_list = []
    for i in range(len(match_data)):
        match = match_data[i]['gameId']
        match_list.append(match)
    return match_list
# 한유저의 매치 ID들을 불러오는 함수
def searchMatchId(accountId):
    match_data = requests.get("https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/" + accountId + '?queue=420',headers=request_headers).json()['matches']
    match_list = []
    for i in range(len(match_data)):
        match = match_data[i]['gameId']
        match_list.append(match)
    return match_list
# 챔피언키로 챔피언 이름 불러오는 함수
def searchChampion(championKey):
    champ = champion_df['name'][champion_df['name'][champion_df['key'] == championKey].index[0]]
    return champ  
# 아이템키로 아이템 이름 불러오는 함수
def searchItem(itemKey):
    item = item_df['name'][item_df['name'][item_df['key'] == itemKey].index[0]]
    return item  

In [7]:
userId, userAccountId, userPuuid = searchUserId('Giyeom') # 유저의 닉네임 Giyeom
match_list = searchChampMatchId(userAccountId, '나르') # 유저의 챔피언 나르로 설정
len(match_list) # 최대가 100개

100

In [8]:
match_list2 = searchMatchId(userAccountId)

In [9]:
match_list[0]

5297081745

In [10]:
# sample_a
a = requests.get('https://kr.api.riotgames.com/lol/match/v4/matches/' + str(match_list[0]), headers=request_headers).json()

In [11]:
list(a)

['gameId',
 'platformId',
 'gameCreation',
 'gameDuration',
 'queueId',
 'mapId',
 'seasonId',
 'gameVersion',
 'gameMode',
 'gameType',
 'teams',
 'participants',
 'participantIdentities']

In [12]:
a['teams']

[{'teamId': 100,
  'win': 'Win',
  'firstBlood': True,
  'firstTower': True,
  'firstInhibitor': False,
  'firstBaron': False,
  'firstDragon': True,
  'firstRiftHerald': True,
  'towerKills': 5,
  'inhibitorKills': 0,
  'baronKills': 0,
  'dragonKills': 2,
  'vilemawKills': 0,
  'riftHeraldKills': 2,
  'dominionVictoryScore': 0,
  'bans': [{'championId': 25, 'pickTurn': 1},
   {'championId': 141, 'pickTurn': 2},
   {'championId': 133, 'pickTurn': 3},
   {'championId': 35, 'pickTurn': 4},
   {'championId': 56, 'pickTurn': 5}]},
 {'teamId': 200,
  'win': 'Fail',
  'firstBlood': False,
  'firstTower': False,
  'firstInhibitor': False,
  'firstBaron': False,
  'firstDragon': False,
  'firstRiftHerald': False,
  'towerKills': 0,
  'inhibitorKills': 0,
  'baronKills': 0,
  'dragonKills': 0,
  'vilemawKills': 0,
  'riftHeraldKills': 0,
  'dominionVictoryScore': 0,
  'bans': [{'championId': 89, 'pickTurn': 6},
   {'championId': 56, 'pickTurn': 7},
   {'championId': 91, 'pickTurn': 8},
   {'ch

In [13]:
a['participants'][0]

{'participantId': 1,
 'teamId': 100,
 'championId': 55,
 'spell1Id': 14,
 'spell2Id': 12,
 'stats': {'participantId': 1,
  'win': True,
  'item0': 4633,
  'item1': 2420,
  'item2': 0,
  'item3': 3020,
  'item4': 1082,
  'item5': 3115,
  'item6': 3364,
  'kills': 10,
  'deaths': 5,
  'assists': 8,
  'largestKillingSpree': 4,
  'largestMultiKill': 2,
  'killingSprees': 2,
  'longestTimeSpentLiving': 363,
  'doubleKills': 1,
  'tripleKills': 0,
  'quadraKills': 0,
  'pentaKills': 0,
  'unrealKills': 0,
  'totalDamageDealt': 76934,
  'magicDamageDealt': 66078,
  'physicalDamageDealt': 9756,
  'trueDamageDealt': 1099,
  'largestCriticalStrike': 0,
  'totalDamageDealtToChampions': 17018,
  'magicDamageDealtToChampions': 14705,
  'physicalDamageDealtToChampions': 1530,
  'trueDamageDealtToChampions': 781,
  'totalHeal': 3294,
  'totalUnitsHealed': 1,
  'damageSelfMitigated': 6903,
  'damageDealtToObjectives': 448,
  'damageDealtToTurrets': 448,
  'visionScore': 15,
  'timeCCingOthers': 1,
  '

In [14]:
a['participantIdentities'][0]

{'participantId': 1,
 'player': {'platformId': 'KR',
  'accountId': 'WjzKtNil9QrMCzNZmg1RmrGSGnb5N-7pGWdTLZkKUX24kW4',
  'summonerName': '게임승패는미드',
  'summonerId': 'Zxn4RuXh6-BBAQYR1Xu05iQ4EGoNg6hlQHvUrjG-gBAbq0g',
  'currentPlatformId': 'KR',
  'currentAccountId': 'WjzKtNil9QrMCzNZmg1RmrGSGnb5N-7pGWdTLZkKUX24kW4',
  'matchHistoryUri': '/v1/stats/player_history/KR/210068988',
  'profileIcon': 4903}}

In [15]:
# sample_a의 유저 닉네임 구하기 
users_info = a['participantIdentities']
user_list = []
for user_info in users_info:
    user_name = user_info['player']['summonerName']
    user_list.append(user_name)

In [16]:
user_list

['게임승패는미드',
 'VC하리',
 '아리장인 고내봉',
 '부산사는송하나',
 '제발나잇값하자좀',
 '프로해보고싶어요',
 '소년장남원딜',
 '감정적인 원숭이',
 '한타천재',
 'Giyeom']

In [17]:
a['participants'][0]['timeline']['lane']

'MIDDLE'

In [18]:
def match_record(matchId):
    match_record = requests.get('https://kr.api.riotgames.com/lol/match/v4/matches/' + str(matchId), headers=request_headers).json()
    users_record = match_record['participants']
    users_info = match_record['participantIdentities']
    user_list = []
    for user_info in users_info:
        user_name = user_info['player']['summonerName']
        user_list.append(user_name)
    users_champion = []
    users_kda = []
    users_items = []
    users_lane = []
    users_win =[]
    for user_record in users_record:
        user_champion = searchChampion(user_record['championId'])
        user_kda = str(user_record['stats']['kills']) + '/' + str(user_record['stats']['deaths']) + '/' + str(user_record['stats']['assists'])
        user_items = []
        for i in range(6):
            itemKey = user_record['stats']['item'+str(i)]
            if itemKey == 0:
                pass
            else:
                user_items.append(searchItem(user_record['stats']['item'+str(i)]))
        user_items = ', '.join(user_items)
        user_lane = user_record['timeline']['lane']
        user_win = user_record['stats']['win']
        if user_win == True:
            user_win = '승리'
        else :
            user_win = '패배'
        users_champion.append(user_champion)
        users_kda.append(user_kda)
        users_items.append(user_items)
        users_lane.append(user_lane)
        users_win.append(user_win)
    df = pd.DataFrame({
    'nickname' : user_list,
    'champion' : users_champion,
    'lane' : users_lane,
    'kda' : users_kda,
    'result_items' : users_items,
    'result' : users_win
    })
    return df

In [19]:
# 전적 test 1
match_record(match_list[25])

,nickname,champion,lane,kda,result_items,result
0,FT lSLAND,렉사이,JUNGLE,9/3/10,"자객의 발톱, 스테락의 도전, 수호 천사, 판금 장화",승리
1,남들보다더딘아이,트위스티드 페이트,MIDDLE,5/2/15,"만년서리, 존야의 모래시계, 부패 물약, 메자이의 영혼약탈자, 열정의 검, 명석함의...",승리
2,Giyeom,나르,TOP,5/1/13,"발걸음 분쇄기, 도란의 검, 판금 장화, 란두인의 예언, 루비 수정, 곡괭이",승리
3,어서오세요버스,쓰레쉬,BOTTOM,0/5/13,"강철의 솔라리 펜던트, 화이트록의 갑옷, 제어 와드, 점화석, 신속의 장화, 루비 수정",승리
4,부식먹을나이,이즈리얼,BOTTOM,5/3/6,"도란의 검, 마나무네, 톱날 단검, 정수 약탈자, 콜필드의 전투 망치, 명석함의 아...",승리
5,지우의 강아지,럼블,MIDDLE,7/4/2,"도란의 방패, 망가진 초시계, 모렐로노미콘, 악마의 마법서, 밤의 수확자, 마법사의 신발",패배
6,유노의 고양이,라칸,BOTTOM,1/3/8,"강철의 솔라리 펜던트, 기동력의 장화, 점화석, 타곤 산의 방벽, 제어 와드, 천 갑옷",패배
7,릐제제,그레이브즈,JUNGLE,1/6/4,"돌풍, 충전형 물약, 판금 장화, 톱날 단검, 민첩성의 망토, 제어 와드",패배
8,슈우프륌,트리스타나,BOTTOM,5/3/3,"유령 무희, 마법무효화의 망토, 도란의 검, 크라켄 학살자, 광전사의 군화",패배
9,송군1,사이온,TOP,0/8/5,"비상의 월갑, 도란의 방패, 발걸음 분쇄기, 판금 장화, 쇠사슬 조끼, 루비 수정",패배


In [20]:
# 전적 test 2
match_record(match_list2[5])

,nickname,champion,lane,kda,result_items,result
0,그웬살면잡아줘,사미라,NONE,8/1/5,"징수의 총, 불멸의 철갑궁, 장화, 도란의 검, 롱소드",승리
1,Giyeom,레넥톤,NONE,3/3/3,"선혈포식자, 판금 장화, 롱소드, 도란의 검, 충전형 물약",승리
2,IndianaJones,레오나,NONE,2/0/10,"룬 강철 어깨 갑옷, 강철의 솔라리 펜던트, 제어 와드, 판금 장화, 천 갑옷",승리
3,착한 마음 정글러,다이애나,NONE,2/2/3,"마법공학 로켓 벨트, 마법사의 신발",승리
4,추 호,베인,NONE,1/1/4,"도란의 검, 롱소드, 롱소드, 광전사의 군화, 크라켄 학살자",승리
5,jinjinjinjinjin,킨드레드,NONE,2/4/3,기동력의 장화,패배
6,나가D지겠습니다,세나,NONE,0/5/2,"해로윙 초승달낫, 광휘의 검, 탐식의 망치, 약간 신비한 신발",패배
7,봇에번쩍탑에번쩍,갈리오,NONE,2/1/2,"충전형 물약, 제어 와드, 마법공학 로켓 벨트, 명석함의 아이오니아 장화, 암흑의 ...",패배
8,IWIWI,자크,NONE,1/5/1,"판금 장화, 충전형 물약, 바미의 불씨, 마법무효화의 망토, 천 갑옷",패배
9,Guo zi chan,쉔,NONE,2/1/3,"도란의 방패, 덤불 조끼, 바미의 불씨, 군단의 방패, 헤르메스의 발걸음, 제어 와드",패배


### 트롤러 잡아내기(아이템 팔기 or 같은 아이템 도배)

### 아이템 파는 유저

In [21]:
userId, userAccountId, userPuuid = searchUserId('아이번이니까렝가해주시면안될까요') 
match_list = searchChampMatchId(userAccountId, '아이번')

In [22]:
test_match = match_record(match_list[0])
test_match

,nickname,champion,lane,kda,result_items,result
0,얼룩덜룩한말,그웬,TOP,9/2/11,"존야의 모래시계, 망각의 구, 균열 생성기, 명석함의 아이오니아 장화, 내셔의 이빨...",승리
1,토리5,리 신,JUNGLE,11/3/12,"곡괭이, 선혈포식자, 칠흑의 양날 도끼, 헤르메스의 발걸음, 롱소드",승리
2,불구경 장인,비에고,MIDDLE,6/2/9,"신성한 파괴자, 곡궁, 판금 장화, 도란의 방패, 흡혈의 낫",승리
3,은정곤듀님,징크스,BOTTOM,7/2/10,"도란의 검, 크라켄 학살자, 루난의 허리케인, B.F. 대검, 광전사의 군화",승리
4,내 인생은 고,쓰레쉬,BOTTOM,0/5/7,"강철의 솔라리 펜던트, 화이트록의 갑옷, 점화석, 기동력의 장화",승리
5,IlllIIIIllIlII,칼리스타,BOTTOM,1/5/8,"루난의 허리케인, 불멸의 철갑궁, 도란의 검, 광전사의 군화, 단검",패배
6,아이번이니까렝가해주시면안될까요,아이번,JUNGLE,0/6/9,,패배
7,비타명재,제드,MIDDLE,6/8/2,"드락사르의 황혼검, 징수의 총, 제어 와드, 명석함의 아이오니아 장화, 롱소드, 롱소드",패배
8,LongStrongWard,노틸러스,BOTTOM,6/6/3,"타곤 산의 방벽, 강철의 솔라리 펜던트, 충전형 물약, 가시 갑옷, 판금 장화",패배
9,메우메우귀여움,스웨인,TOP,1/8/3,"도란의 반지, 리안드리의 고뇌, 장화, 망각의 구, 제어 와드",패배


In [23]:
test_match['trolling'] = test_match.apply(lambda r : True if str(r.result_items) == '' or str(r.result_items.split(', ')[0]) == str(r.result_items.split(', ')[-1]) else False, axis=1)

In [24]:
test_match

,nickname,champion,lane,kda,result_items,result,trolling
0,얼룩덜룩한말,그웬,TOP,9/2/11,"존야의 모래시계, 망각의 구, 균열 생성기, 명석함의 아이오니아 장화, 내셔의 이빨...",승리,False
1,토리5,리 신,JUNGLE,11/3/12,"곡괭이, 선혈포식자, 칠흑의 양날 도끼, 헤르메스의 발걸음, 롱소드",승리,False
2,불구경 장인,비에고,MIDDLE,6/2/9,"신성한 파괴자, 곡궁, 판금 장화, 도란의 방패, 흡혈의 낫",승리,False
3,은정곤듀님,징크스,BOTTOM,7/2/10,"도란의 검, 크라켄 학살자, 루난의 허리케인, B.F. 대검, 광전사의 군화",승리,False
4,내 인생은 고,쓰레쉬,BOTTOM,0/5/7,"강철의 솔라리 펜던트, 화이트록의 갑옷, 점화석, 기동력의 장화",승리,False
5,IlllIIIIllIlII,칼리스타,BOTTOM,1/5/8,"루난의 허리케인, 불멸의 철갑궁, 도란의 검, 광전사의 군화, 단검",패배,False
6,아이번이니까렝가해주시면안될까요,아이번,JUNGLE,0/6/9,,패배,True
7,비타명재,제드,MIDDLE,6/8/2,"드락사르의 황혼검, 징수의 총, 제어 와드, 명석함의 아이오니아 장화, 롱소드, 롱소드",패배,False
8,LongStrongWard,노틸러스,BOTTOM,6/6/3,"타곤 산의 방벽, 강철의 솔라리 펜던트, 충전형 물약, 가시 갑옷, 판금 장화",패배,False
9,메우메우귀여움,스웨인,TOP,1/8/3,"도란의 반지, 리안드리의 고뇌, 장화, 망각의 구, 제어 와드",패배,False


### 같은 아이템 도배하는 유저

In [25]:
userId, userAccountId, userPuuid = searchUserId('케이온') 
match_list = searchChampMatchId(userAccountId, '나서스')
test_match2 = match_record(match_list[0])
test_match2

,nickname,champion,lane,kda,result_items,result
0,먹삼돌이,룰루,NONE,1/1/0,"타곤 산의 방패, 장화, 제어 와드, 체력 물약, 루비 수정, 밴들유리 거울",패배
1,하울링 404,리 신,NONE,1/3/2,"요정의 부적, 요정의 부적, 요정의 부적, 요정의 부적, 요정의 부적, 요정의 부적",패배
2,케이온,나서스,NONE,1/1/1,"원기 회복의 구슬, 원기 회복의 구슬, 원기 회복의 구슬, 원기 회복의 구슬, 원기...",패배
3,다이아는신비롭다,에코,NONE,1/3/0,"암흑의 인장, 충전형 물약, 마법사의 신발, 마법공학 교류 발전기, 루비 수정, 방...",패배
4,할아버지의롤렉스,바루스,NONE,0/3/1,"톱날 단검, 충전형 물약, 여신의 눈물, 도란의 방패, 롱소드, 명석함의 아이오니아 장화",패배
5,딜 서폿이면 탑감,이즈리얼,NONE,2/0/3,"제어 와드, 콜필드의 전투 망치, 여신의 눈물, 신성한 파괴자, 도란의 검, 명석함...",승리
6,니칼라똥파워,카서스,NONE,4/0/4,"제어 와드, 충전형 물약, 초시계, 암흑의 인장, 리안드리의 고뇌, 마법사의 신발",승리
7,rnldyalyuqi,카르마,NONE,1/1/7,"얼음 정수의 파편, 슈렐리아의 군가, 제어 와드, 신속의 장화, 충전형 물약",승리
8,Inversion user,다리우스,NONE,3/2/0,"루비 수정, 발걸음 분쇄기, 충전형 물약, 판금 장화, 도란의 검",승리
9,오늘 넌 뭐했어,피오라,NONE,1/1/1,"충전형 물약, 선혈포식자, 도란의 검, 장화",승리


In [26]:
test_match2['trolling'] = test_match2.apply(lambda r : True if str(r.result_items) == '' or str(r.result_items.split(', ')[0]) == str(r.result_items.split(', ')[-1]) else False, axis=1)
test_match2

,nickname,champion,lane,kda,result_items,result,trolling
0,먹삼돌이,룰루,NONE,1/1/0,"타곤 산의 방패, 장화, 제어 와드, 체력 물약, 루비 수정, 밴들유리 거울",패배,False
1,하울링 404,리 신,NONE,1/3/2,"요정의 부적, 요정의 부적, 요정의 부적, 요정의 부적, 요정의 부적, 요정의 부적",패배,True
2,케이온,나서스,NONE,1/1/1,"원기 회복의 구슬, 원기 회복의 구슬, 원기 회복의 구슬, 원기 회복의 구슬, 원기...",패배,True
3,다이아는신비롭다,에코,NONE,1/3/0,"암흑의 인장, 충전형 물약, 마법사의 신발, 마법공학 교류 발전기, 루비 수정, 방...",패배,False
4,할아버지의롤렉스,바루스,NONE,0/3/1,"톱날 단검, 충전형 물약, 여신의 눈물, 도란의 방패, 롱소드, 명석함의 아이오니아 장화",패배,False
5,딜 서폿이면 탑감,이즈리얼,NONE,2/0/3,"제어 와드, 콜필드의 전투 망치, 여신의 눈물, 신성한 파괴자, 도란의 검, 명석함...",승리,False
6,니칼라똥파워,카서스,NONE,4/0/4,"제어 와드, 충전형 물약, 초시계, 암흑의 인장, 리안드리의 고뇌, 마법사의 신발",승리,False
7,rnldyalyuqi,카르마,NONE,1/1/7,"얼음 정수의 파편, 슈렐리아의 군가, 제어 와드, 신속의 장화, 충전형 물약",승리,False
8,Inversion user,다리우스,NONE,3/2/0,"루비 수정, 발걸음 분쇄기, 충전형 물약, 판금 장화, 도란의 검",승리,False
9,오늘 넌 뭐했어,피오라,NONE,1/1/1,"충전형 물약, 선혈포식자, 도란의 검, 장화",승리,False


### 최근 10게임 전적구하기

In [30]:
user = '케이온'
userId, userAccountId, userPuuid = searchUserId(user)
match_list = searchMatchId(userAccountId)[:30] # 30개 불러오기

In [31]:
# 유저 인덱스 구하기
def userIndex(users_info):
    user_list = []
    for user_info in users_info:
        user_name = user_info['player']['summonerName']
        user_list.append(user_name)
    user_index = user_list.index(user)
    return user_index

In [32]:
users_champion = []
users_kda = []
users_items = []
users_lane = []
users_win =[]
for match in match_list:
    match_info = requests.get('https://kr.api.riotgames.com/lol/match/v4/matches/' + str(match), headers=request_headers).json()
    users_info = match_info['participantIdentities']
    user_index = userIndex(users_info)
    user_record = match_info['participants'][user_index]
    user_champion = searchChampion(user_record['championId'])
    user_kda = str(user_record['stats']['kills']) + '/' + str(user_record['stats']['deaths']) + '/' + str(user_record['stats']['assists'])
    user_items = []
    for i in range(6):
        itemKey = user_record['stats']['item'+str(i)]
        if itemKey == 0:
            pass
        else:
            user_items.append(searchItem(user_record['stats']['item'+str(i)]))
    user_items = ', '.join(user_items)
    user_lane = user_record['timeline']['lane']
    user_win = user_record['stats']['win']
    if user_win == True:
            user_win = '승리'
    else :
        user_win = '패배'
    users_champion.append(user_champion)
    users_kda.append(user_kda)
    users_items.append(user_items)
    users_lane.append(user_lane)
    users_win.append(user_win)
    
df = pd.DataFrame({
'champion' : users_champion,
'lane' : users_lane,
'kda' : users_kda,
'result_items' : users_items,
'result' : users_win
})
df['trolling'] = df.apply(lambda r : True if str(r.result_items) == '' or str(r.result_items.split(', ')[0]) == str(r.result_items.split(', ')[-1]) else False, axis=1)
df

,champion,lane,kda,result_items,result,trolling
0,말파이트,TOP,1/5/12,"부패 물약, 덤불 조끼, 증오의 사슬, 판금 장화, 태양불꽃 방패, 루비 수정",승리,False
1,가렌,NONE,4/1/1,"도란의 방패, 충전형 물약, 헤르메스의 발걸음, 태양불꽃 방패, 루비 수정",승리,False
2,가렌,TOP,3/2/7,"도란의 방패, 태양불꽃 방패, 판금 장화, 망자의 갑옷, 덤불 조끼",승리,False
3,가렌,TOP,1/5/6,"점화석, 태양불꽃 방패, 망자의 갑옷, 판금 장화, 덤불 조끼, 대자연의 힘",승리,False
4,나서스,NONE,1/1/1,"원기 회복의 구슬, 원기 회복의 구슬, 원기 회복의 구슬, 원기 회복의 구슬, 원기...",패배,True
5,마오카이,MIDDLE,4/7/11,"가시 갑옷, 태양불꽃 방패, 판금 장화, 증오의 사슬, 음전자 망토",승리,False
6,가렌,NONE,2/2/0,"도란의 방패, 바미의 불씨, 판금 장화, 망자의 갑옷, 점화석, 거인의 허리띠",패배,False
7,마오카이,MIDDLE,2/3/4,"부패 물약, 태양불꽃 방패, 판금 장화, 덤불 조끼, 점화석, 루비 수정",패배,False
8,마오카이,TOP,1/4/1,"도란의 방패, 덤불 조끼, 판금 장화, 터보 화공 탱크, 초시계, 천 갑옷",패배,False
9,마오카이,MIDDLE,4/6/13,"부패 물약, 헤르메스의 발걸음, 가시 갑옷, 태양불꽃 방패, 음전자 망토, 거인의 허리띠",패배,False
